# Viator End of Year Analytics

In [ ]:
import sys
sys.path.append('../../')
from env import conn

In [ ]:
import pandas as pd
import numpy as np
from env import conn
import matplotlib as plt
import seaborn as sns
%matplotlib inline
cs = conn.cursor()

In [ ]:
def return_df(query):
    cs = conn.cursor()
    query = query
    result = cs.execute(query)
    df = result.fetch_pandas_all()
    cs.close()
    return df

In [ ]:
lc_daily_metrics = return_df(
    """select * from METRICS.LOYALTY_CARDS.LC__LINKS_JOINS__DAILY_RETAILER
        where loyalty_plan_company = 'Viator'
        order by date;    
    """
        )

txn_daily_metrics = return_df(
    """select * from METRICS.TRANSACTIONS.TRANS__TRANS__DAILY_RETAILER
        where loyalty_plan_company = 'Viator'
         order by date;    
    """
        )

user_level_txns = return_df(
    """select * from PROD.BINK_SECURE.FACT_TRANSACTION
        where loyalty_plan_company = 'Viator'
         order by event_date_time;    
    """
        )